In [1]:
# !pip install datasets
# !pip install torchmetrics
# !pip install pytz
# !pip install persiantools
# !pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mount Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# %cd /content/drive/MyDrive

/content/drive/MyDrive


# Configuration

In [4]:
config = {}
config['root_path'] = '/home/user01/'
config['series_ID'] = 103
config['series_desc'] = '''
targeted backdoor attack labels changed - LR influence attack manually implemented (without the help of jax)
'''
config['log_path'] = config['root_path']+'experiments/CIFAR10/SVM/'
config['log'] = {}

In [5]:
config['poisoning_rate'] = 1.0
# config['num_clean_examples'] = 200
config['learning_rate'] = 0.01
config['batch_size'] = 32
config['num_epochs'] = 50

In [6]:
config['log']['model'] = 'SVM'
config['log']['dataset'] = 'CIFAR10(0-6)'
config['log']['task'] = 'binary classification'
config['log']['pytorch_seed'] = 50
config['log']['numpy_seed'] = 50
config['log']['attack'] = 'label-flip'
config['log']['method'] = 'modify'

In [7]:
config['log_path'] += (str(config['series_ID']) + '-' + config['log']['attack'] + '-' + config['log']['dataset'] + '-' + str(int(config['poisoning_rate']*100)) + '-' + config['log']['method'] + '.json').lower().replace(' ', '')

In [8]:
config['log_path']

'/content/drive/MyDrive/experiments/CIFAR10/SVM/103-label-flip-cifar10(0-6)-100-modify.json'

In [9]:
### WE NEED THIS TO IMPORT THE NECESSARY LIBRARIES ###
import sys
sys.path.append(config['root_path'])

In [10]:
import datascience, poisoning, report
from datascience.data import CIFAR10, MNIST, IMDB, BOSTON
from datascience.general import train_dev_test_split, join_np_arrays, describe_dataset
from poisoning.process import attacker, defender, label_flip_attacker
from poisoning.eval import attack_success_rate, benign_accuracy, test_accuracy
from report.log import JSONLogger, TextLogger, tehran_datetime
from temporary.functions import _reload
import numpy as np
import torch
from torchmetrics import HingeLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd.functional import hessian, jacobian
from torch.autograd import grad
from torch.nn.utils import _stateless
from datetime import datetime
import pytz
from persiantools.jdatetime import JalaliDate

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [11]:
# _reload(poisoning.process)
# _reload(poisoning.eval)
# _reload(datascience.data)
# _reload(datascience.general)
# _reload(report.log)

In [12]:
torch.manual_seed(config['log']['pytorch_seed'])

# Loading a Dataset

In [13]:
dataset = CIFAR10()
dataset.select_labels([0,6], 'train')
dataset.select_labels([0,6], 'test')
dataset.change_labels({6:1})
# dataset.resize(10,10)
# dataset.change_labels({0:-1})
dataset.flatten()

# Attack

In [14]:
att = label_flip_attacker(dataset.x_train, dataset.y_train, {0:1,1:0}, config['poisoning_rate'], config['log']['method'], config['log']['numpy_seed'])

In [15]:
att.attack()

In [16]:
result = att.return_aggregated_result()

In [17]:
config['data-train'] = describe_dataset(result['x_train'], result['y_train'], 'training dataset')

In [18]:
config['data-test'] = describe_dataset(dataset.x_test, dataset.y_test, 'testing dataset')

# Prepare the Data

In [19]:
class MyVectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]).to(device), torch.Tensor(self.labels[idx]).to(device)

In [20]:
train_dataset = MyVectorDataset(result['x_train'], result['y_train'])
test_dataset = MyVectorDataset(dataset.x_test, dataset.y_test)
# clean_dataset = MyVectorDataset(dataset.x_train_clean, dataset.y_train_clean)

In [21]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
# clean_dataloader = DataLoader(clean_dataset, batch_size=config['batch_size'], shuffle=True)

# Train a Model

In [22]:
class SVM(torch.nn.Module):
    def __init__(self):
        super(SVM, self).__init__() 
        self.linear = torch.nn.Linear(in_features=3072, out_features=1, bias=True)
    def forward(self, x):
        output = self.linear(x)
        return output

In [23]:
def output_to_label(out):
    if out >= 0:
      return 1
    else:
      return 0

In [24]:
def make_prediction(model, x_arr):
  outs = list(model(torch.Tensor(x_arr)).squeeze().detach().numpy())
  labels = [output_to_label(out) for out in outs]
  return np.array(labels)

In [25]:
model = SVM()
model = model.to(device)
loss_fn = HingeLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)

In [26]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")

In [27]:
def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.cpu().numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.cpu().numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    config['log']['accuracy_'+name] = (100*correct)
    config['log']['loss_'+name] = sum_test_loss
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [28]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch_num)

 Epoch 50 - loss: 4233.364258  [ 9600/10000] 

In [29]:
test_loop(train_dataloader, model, loss_fn, config['num_epochs'], 'Train')

 Epoch 50 - Train Error: Accuracy: 89.8%, Avg loss: 98.085961 

In [30]:
test_loop(test_dataloader, model, loss_fn, config['num_epochs'], 'Test')

 Epoch 50 - Test Error: Accuracy: 11.3%, Avg loss: 2378.075172 

In [31]:
y_train_pred = make_prediction(model, result['x_train'])
y_test_pred = make_prediction(model, dataset.x_test)

In [32]:
config['log']['benign_accuracy'] = benign_accuracy(result['y_train'], y_train_pred, result['is_poison'])
config['log']['attack_success_rate'] = attack_success_rate(result['y_train'], y_train_pred, result['is_poison'])

In [33]:
config['datetime'] = tehran_datetime()

In [34]:
config

{'series_ID': 103,
 'series_desc': '\ntargeted backdoor attack labels changed - LR influence attack manually implemented (without the help of jax)\n',
 'log_path': '/content/drive/MyDrive/experiments/CIFAR10/SVM/103-label-flip-cifar10(0-6)-100-modify.json',
 'log': {'model': 'SVM',
  'dataset': 'CIFAR10(0-6)',
  'task': 'binary classification',
  'pytorch_seed': 50,
  'numpy_seed': 50,
  'attack': 'label-flip',
  'method': 'modify',
  'accuracy_Train': 89.77000000000001,
  'loss_Train': 98.0859606103897,
  'accuracy_Test': 11.3,
  'loss_Test': 2378.075171875,
  'benign_accuracy': 0.0,
  'attack_success_rate': 0.8977},
 'poisoning_rate': 1.0,
 'learning_rate': 0.01,
 'batch_size': 32,
 'num_epochs': 50,
 'data-train': {'name': 'training dataset',
  'num_samples': 10000,
  'num_features': (3072,),
  'class_count': {0: 5000, 1: 5000}},
 'data-test': {'name': 'testing dataset',
  'num_samples': 2000,
  'num_features': (3072,),
  'class_count': {0: 1000, 1: 1000}},
 'datetime': '1401-08-09 

In [35]:
logger = JSONLogger(config['log_path'], config)
logger.log()